In [ ]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import glob
import os
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.cluster import KMeans
import tensorflow as tf

In [ ]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt


class Hog_descriptor():
    def __init__(self, img, cell_size=16, bin_size=8):
        self.img = img
        self.img = np.sqrt(img / float(np.max(img)))
        self.img = self.img * 255
        self.cell_size = cell_size
        self.bin_size = bin_size
        self.angle_unit = 360 / self.bin_size
        assert type(self.bin_size) == int, "bin_size should be integer,"
        assert type(self.cell_size) == int, "cell_size should be integer,"
        assert type(self.angle_unit) == int, "bin_size should be divisible by 360"

    def extract(self):
        height, width = self.img.shape
        gradient_magnitude, gradient_angle = self.global_gradient()
        gradient_magnitude = abs(gradient_magnitude)
        cell_gradient_vector = np.zeros((height / self.cell_size, width / self.cell_size, self.bin_size))
        for i in range(cell_gradient_vector.shape[0]):
            for j in range(cell_gradient_vector.shape[1]):
                cell_magnitude = gradient_magnitude[i * self.cell_size:(i + 1) * self.cell_size,
                                 j * self.cell_size:(j + 1) * self.cell_size]
                cell_angle = gradient_angle[i * self.cell_size:(i + 1) * self.cell_size,
                             j * self.cell_size:(j + 1) * self.cell_size]
                cell_gradient_vector[i][j] = self.cell_gradient(cell_magnitude, cell_angle)

        hog_image = self.render_gradient(np.zeros([height, width]), cell_gradient_vector)
        hog_vector = []
        for i in range(cell_gradient_vector.shape[0] - 1):
            for j in range(cell_gradient_vector.shape[1] - 1):
                block_vector = []
                block_vector.extend(cell_gradient_vector[i][j])
                block_vector.extend(cell_gradient_vector[i][j + 1])
                block_vector.extend(cell_gradient_vector[i + 1][j])
                block_vector.extend(cell_gradient_vector[i + 1][j + 1])
                mag = lambda vector: math.sqrt(sum(i ** 2 for i in vector))
                magnitude = mag(block_vector)
                if magnitude != 0:
                    normalize = lambda block_vector, magnitude: [element / magnitude for element in block_vector]
                    block_vector = normalize(block_vector, magnitude)
                hog_vector.append(block_vector)
        return hog_vector, hog_image

    def global_gradient(self):
        gradient_values_x = cv2.Sobel(self.img, cv2.CV_64F, 1, 0, ksize=5)
        gradient_values_y = cv2.Sobel(self.img, cv2.CV_64F, 0, 1, ksize=5)
        gradient_magnitude = cv2.addWeighted(gradient_values_x, 0.5, gradient_values_y, 0.5, 0)
        gradient_angle = cv2.phase(gradient_values_x, gradient_values_y, angleInDegrees=True)
        return gradient_magnitude, gradient_angle

    def cell_gradient(self, cell_magnitude, cell_angle):
        orientation_centers = [0] * self.bin_size
        for i in range(cell_magnitude.shape[0]):
            for j in range(cell_magnitude.shape[1]):
                gradient_strength = cell_magnitude[i][j]
                gradient_angle = cell_angle[i][j]
                min_angle, max_angle, mod = self.get_closest_bins(gradient_angle)
                orientation_centers[min_angle] += (gradient_strength * (1 - (mod / self.angle_unit)))
                orientation_centers[max_angle] += (gradient_strength * (mod / self.angle_unit))
        return orientation_centers

    def get_closest_bins(self, gradient_angle):
        idx = int(gradient_angle / self.angle_unit)
        mod = gradient_angle % self.angle_unit
        if idx == self.bin_size:
            return idx - 1, (idx) % self.bin_size, mod
        return idx, (idx + 1) % self.bin_size, mod

    def render_gradient(self, image, cell_gradient):
        cell_width = self.cell_size / 2
        max_mag = np.array(cell_gradient).max()
        for x in range(cell_gradient.shape[0]):
            for y in range(cell_gradient.shape[1]):
                cell_grad = cell_gradient[x][y]
                cell_grad /= max_mag
                angle = 0
                angle_gap = self.angle_unit
                for magnitude in cell_grad:
                    angle_radian = math.radians(angle)
                    x1 = int(x * self.cell_size + magnitude * cell_width * math.cos(angle_radian))
                    y1 = int(y * self.cell_size + magnitude * cell_width * math.sin(angle_radian))
                    x2 = int(x * self.cell_size - magnitude * cell_width * math.cos(angle_radian))
                    y2 = int(y * self.cell_size - magnitude * cell_width * math.sin(angle_radian))
                    cv2.line(image, (y1, x1), (y2, x2), int(255 * math.sqrt(magnitude)))
                    angle += angle_gap
        return image


In [ ]:
X=[]

labels=[]
j=-1
k=0
for dance_dir_path in glob.glob("/home/rishotics/ppd_sir/dataset/*"):
    dance_label=dance_dir_path.split("/")[-1]
    dance_feature=[]
    j=j+1
    for dance_dir_path_vnum in glob.glob("/home/rishotics/ppd_sir/dataset/"+dance_label+"/*"):
        video_num=dance_dir_path_vnum.split("/")[-1]
        video_num=video_num[1:]
        
        dance_image=[]
        for i,dance_path_image in enumerate(glob.glob(os.path.join(dance_dir_path_vnum,"*.jpg"))):
            
            if(i<2500):
                
                k=k+1
                image=cv.imread(dance_path_image,cv2.IMREAD_GRAYSCALE)
                image = cv.resize(image, (64, 128))
                image[0,0]=image[0,0]+1
                
                hog = Hog_descriptor(image, cell_size=8, bin_size=9)
                from numpy import *
                where_are_NaNs = isnan(image)
                vector, image = hog.extract()
                image=np.reshape(np.array(vector),-1)
                
                
                dance_image.append(image)
                labels=np.append(labels,j)
                
            i=1+i
            
        labels=np.array(labels)    
        dance_feature.append(np.array(dance_image))
        print(np.array(dance_feature).shape)
    #dance_feature_BN=BN(dance_feature)
    X.append(dance_feature)
    print(np.array(X).shape)
   
X=np.array(X)    

dance_to_dict={i:k for k,i in enumerate(np.unique(labels))}
dict_to_id={i:k for k,i in dance_to_dict.items()}
label_id=np.array([dance_to_dict[x] for x in labels]) 
            
        
        

In [ ]:
X_train=np.array(X[0])
for i,j in enumerate(X):
    if (i != 0):
        
        if(np.array(j).shape[0]==10):
            print("1")
            X_train=np.concatenate((X_train,np.array(j)),axis=0)

a,b,c=X_train.shape
X_train=np.reshape(X_train,(a*b,c))
print(X_train.shape)

labels.shape
X_trai, X_test, y_train, y_test = train_test_split(X_train, labels, test_size=0.33, random_state=42)
lin_clf = svm.LinearSVC()
lin_clf.fit(X_trai,y_train) 
100*lin_clf.score(X_test,y_test)